<a href="https://colab.research.google.com/github/sayanbanerjee32/keyword_search/blob/main/Semantic_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Word vectors and SIF

In [1]:
%%capture
# To import kaggle datasets
!pip install --upgrade pip 
!pip install kaggle==1.5.6

In [2]:
# # import colab libraries
# from google.colab import files
# # import kaggle json to connect to kaggle user account to download datsets
# files.upload()

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so lets move it there.
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/Kaggle_api_auth/kaggle.json ~/.kaggle/

In [4]:
# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
# download our dataset
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews -p 'datasets'

 66% 17.0M/25.7M [00:01<00:00, 14.3MB/s]
100% 25.7M/25.7M [00:01<00:00, 18.1MB/s]


In [6]:
#unzip
!unzip datasets/imdb-dataset-of-50k-movie-reviews.zip -d datasets/

Archive:  datasets/imdb-dataset-of-50k-movie-reviews.zip
  inflating: datasets/IMDB Dataset.csv  


## Sentence BERT
https://evergreenllc2020.medium.com/semantic-search-engine-with-s-abbfb3cd9377

In [7]:
%%capture
!pip install sentence-transformers

In [18]:
import scipy

In [8]:
from sentence_transformers import SentenceTransformer
# Load the BERT model. Various models trained on Natural Language Inference (NLI) https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/nli-models.md and 
# Semantic Textual Similarity are available https://github.com/UKPLab/sentence-transformers/blob/master/docs/pretrained-models/sts-models.md

model = SentenceTransformer('bert-base-nli-mean-tokens')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

### Load data and create DF

In [10]:
import pandas as pd
df = pd.read_csv('datasets/IMDB Dataset.csv')
df.shape


(50000, 2)

In [11]:
df.head(1)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive


### Preprocess and tokenise

In [14]:
import re
def fix_text(txt):
    return re.sub(r'(<br />)+','<LINEBREAK>',txt)
    # return clean(txt,
    #             fix_unicode=True,               # fix various unicode errors
    #             to_ascii=True,                  # transliterate to closest ASCII representation
    #             lower=True,                     # lowercase text
    #             no_line_breaks=True,           # fully strip line breaks as opposed to only normalizing them
    #             no_urls=False,                  # replace all URLs with a special token
    #             no_emails=True,                # replace all email addresses with a special token
    #             no_phone_numbers=True,         # replace all phone numbers with a special token
    #             no_numbers=True,               # replace all numbers with a special token
    #             no_digits=False,                # replace all digits with a special token
    #             no_currency_symbols=True,      # replace all currency symbols with a special token
    #             no_punct=True,                 # remove punctuations
    #             replace_with_punct="<PUNCT>",          # instead of removing punctuations you may replace them
    #             replace_with_url="<URL>",
    #             replace_with_email="<EMAIL>",
    #             replace_with_phone_number="<PHONE>",
    #             replace_with_number="<NUMBER>",
    #             replace_with_digit="0",
    #             replace_with_currency_symbol="<CUR>",
    #             lang="en"                       # set to 'de' for German special handling
    #         )

In [15]:
# nlp = spacy.load("en_core_web_sm")
# tok_text=[] # for our tokenised corpus
text = df.review.tolist()
text = [fix_text(str(i)) for i in text]
# spacy = WordTokenizer()
# toks = first(spacy([text]))
# print(coll_repr(toks, 30))

In [16]:
# Each sentence is encoded as a 1-D vector with 78 columns
sentence_embeddings = model.encode(text)
print('Sample BERT embedding vector - length', len(sentence_embeddings[0]))
print('Sample BERT embedding vector - note includes negative values', sentence_embeddings[0])

Sample BERT embedding vector - length 768
Sample BERT embedding vector - note includes negative values [-5.08118346e-02  5.95830441e-01  1.99803323e-01  2.78222144e-01
 -4.52101976e-01 -1.08036768e+00  1.03214169e+00  3.52091759e-01
  3.40097487e-01  3.90370518e-01 -2.32599050e-01  6.40658498e-01
 -9.81679931e-02  1.30183786e-01 -4.80243742e-01  6.40568674e-01
 -9.97826234e-02 -4.45801497e-01  2.82441467e-01 -4.27952945e-01
  6.38508350e-02 -8.11117768e-01  1.31785214e-01 -1.91828743e-01
  9.30946589e-01  6.46902263e-01  1.91874564e-01 -1.15187451e-01
 -1.02817416e+00  6.52789354e-01  1.07797980e-01  4.89047766e-01
 -7.18224823e-01 -2.85665095e-01 -2.44571075e-01  1.06919849e+00
  6.66620433e-02  1.00747108e-01  4.16211993e-01 -1.35067701e-01
  4.08418089e-01  1.55033059e-02  7.18979985e-02 -7.64394641e-01
 -7.26450562e-01 -3.28554332e-01  1.13498139e+00  2.04159707e-01
  5.58873415e-01 -5.28515577e-01  4.85164583e-01  5.86391330e-01
  2.28648692e-01 -3.46608758e-01  2.43186414e-01 -6.

In [21]:
#@title Sematic Search Form

# code adapted from https://github.com/UKPLab/sentence-transformers/blob/master/examples/application_semantic_search.py

#query = 'bad weather' #@param {type: 'string'}
#query = 'employee stop working' #@param {type: 'string'}
query = "Security State Penitentary"
#query = 'global warming impact'
#query = 'wildfires in australia'

queries = [query]
query_embeddings = model.encode(queries)

# Find the closest 3 sentences of the corpus for each query sentence based on cosine similarity
number_top_matches = 3 #@param {type: "number"}

print("Semantic Search Results")

for query, query_embedding in zip(queries, query_embeddings):
    distances = scipy.spatial.distance.cdist([query_embedding], sentence_embeddings, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])
    print(type(results))

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for idx, distance in results[0:number_top_matches]:
        print(text[idx].strip(), "(Cosine Score: %.4f)" % (1-distance))

Semantic Search Results
<class 'list'>




Query: Security State Penitentary

Top 5 most similar sentences in corpus:
North And South (1985): Patrick Swayze, James Read, Lesley Anne Down, Wendy Kilbourne, Terri Garber, Kirstie Alley, Genie Francis, Phillip Casnoff, Jean Simmons, John Stockwell, Lewis Smith, David Carradine, Inga Swenson, Jonathan Frakes, Wendy Fulton, Erica Gimpel, Tony Frank, Jim Metzler, Olivia Cole, Andy Stahl, William Ostrander, George Stanford Brown, Robert Mitchum, Morgain Fairchild, Johnny Cash, Hal Holbrook, Gene Kelly, David Ogden Stiers, John Anderson, Lee Bergere, Olivia De Havilland, Elizabeth Taylor, Forest Whitaker, Robert Jones, ....Director Richard T. Heffron, Teleplay...Paul F. Edwards, Patricia Green, Douglas Heyes, Kathleen A. Shelley.<LINEBREAK>Based on John Jake's successful paperback novels "North and South", "Love and War" and "Heaven and Hell", this was a mini series on television from 1985 to 1987. Its success owed more to the success of "Roots